In [58]:
import forallpeople as si
import numpy as np
from IPython.display import display, Markdown, clear_output, HTML
import ipywidgets as widgets

si.environment("default")

# Add custom CSS for better styling
display(
    HTML(
        """
<style>
.calculator-container {
    font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Arial,
    sans-serif;
}
.calculator-header {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    padding: 12px 16px;
    border-radius: 8px;
    margin-bottom: 12px;
    box-shadow: 0 2px 4px rgba(0,0,0,0.1);
}
.section-header {
    color: #667eea;
    border-bottom: 2px solid #667eea;
    padding-bottom: 4px;
    margin-top: 12px;
    margin-bottom: 8px;
    font-weight: 600;
    font-size: 1.1em;
}
.param-line {
    padding: 2px 0;
    padding-left: 20px;
    line-height: 1.4;
    font-size: 0.95em;
    max-width: 100%;
    overflow-wrap: break-word;
    word-wrap: break-word;
}
.result-line {
    padding: 4px 8px;
    padding-left: 20px;
    background: #f8f9fa;
    margin: 2px 0;
    border-radius: 4px;
    border-left: 3px solid #667eea;
    font-size: 0.95em;
    line-height: 1.4;
    max-width: 100%;
    overflow-wrap: break-word;
    word-wrap: break-word;
    box-sizing: border-box;
}
/* Slider Styling - Multiple selectors for compatibility */
.widget-slider .widget-label {
    font-weight: 600;
    color: #333;
    font-size: 0.95em;
}

/* Target the slider track */
.widget-slider .slider,
.widget-hslider .slider,
div.ui-slider,
.ui-widget-content {
    height: 4px !important;
    background: #e0e0e0 !important;
    border-radius: 2px !important;
}

/* Target the slider handle/knob */
.widget-slider .ui-slider-handle,
.widget-hslider .ui-slider-handle,
.ui-slider-handle,
.ui-state-default,
span.ui-slider-handle {
    width: 14px !important;
    height: 14px !important;
    border-radius: 50% !important;
    background: #dc3545 !important;
    background-color: #dc3545 !important;
    border: 2px solid white !important;
    box-shadow: 0 1px 3px rgba(0,0,0,0.2) !important;
    cursor: pointer !important;
    top: -5px !important;
}

.widget-slider .ui-slider-handle:hover,
.ui-slider-handle:hover {
    background: #c82333 !important;
    background-color: #c82333 !important;
    box-shadow: 0 2px 4px rgba(0,0,0,0.3) !important;
}

/* Target the filled slider range */
.widget-slider .ui-slider-range,
.widget-hslider .ui-slider-range,
.ui-slider-range,
.ui-widget-header,
div.ui-slider-range {
    background: #dc3545 !important;
    background-color: #dc3545 !important;
    border-radius: 2px !important;
}

.widget-readout {
    font-weight: 600;
    color: #333;
    font-size: 0.9em;
}
.widget-box > * {
    margin-bottom: 8px !important;
}
.widget-output {
    overflow-x: hidden !important;
    max-width: 100% !important;
}
.jp-OutputArea-output {
    overflow-x: hidden !important;
}
body {
    overflow-x: hidden !important;
}
</style>
"""
    )
)

# Create interactive widgets for each input parameter
eta_slider = widgets.FloatSlider(
    value=0.9,
    min=0.5,
    max=1.0,
    step=0.01,
    description="η:",
    continuous_update=False,
    style={"description_width": "150px", "handle_color": "#20c997"},
    layout=widgets.Layout(width="100%"),
)

n_slider = widgets.IntSlider(
    value=4,
    min=1,
    max=4,
    step=1,
    description="n:",
    continuous_update=False,
    style={"description_width": "150px", "handle_color": "#20c997"},
    layout=widgets.Layout(width="100%"),
)

alpha_B_slider = widgets.FloatSlider(
    value=0.7,
    min=0.1,
    max=0.9,
    step=0.05,
    description="αB:",
    continuous_update=False,
    style={"description_width": "150px", "handle_color": "#20c997"},
    layout=widgets.Layout(width="100%"),
)

p_backup_slider = widgets.FloatSlider(
    value=25,
    min=1,
    max=100,
    step=1,
    description="P_BACKUP (W):",
    continuous_update=False,
    style={"description_width": "150px", "handle_color": "#20c997"},
    layout=widgets.Layout(width="100%"),
)

t_backup_slider = widgets.FloatSlider(
    value=1.8,
    min=0.1,
    max=10,
    step=0.1,
    description="t_BACKUP (s):",
    continuous_update=False,
    style={"description_width": "150px", "handle_color": "#20c997"},
    layout=widgets.Layout(width="100%"),
)

v_cell_max_slider = widgets.FloatSlider(
    value=2.5,
    min=1.0,
    max=5.0,
    step=0.1,
    description="V_CELL(MAX) (V):",
    continuous_update=False,
    style={"description_width": "150px", "handle_color": "#20c997"},
    layout=widgets.Layout(width="100%"),
)

r_snsc_slider = widgets.FloatSlider(
    value=0.006,
    min=0.001,
    max=0.02,
    step=0.001,
    description="R_SNSC (Ω):",
    continuous_update=False,
    readout_format=".4f",
    style={"description_width": "150px", "handle_color": "#20c997"},
    layout=widgets.Layout(width="100%"),
)

# Output widget to display results
output = widgets.Output(layout=widgets.Layout(overflow_x="hidden"))


def calculate_and_display(
    eta, n, alpha_B, p_backup, t_BACKUP, v_cell_max, r_snsc
):
    """Perform calculations and display results"""
    with output:
        clear_output(wait=True)

        # Convert to SI units
        p_backup_si = p_backup * si.W
        t_BACKUP_si = t_BACKUP * si.s
        v_cell_max_si = v_cell_max * si.V
        r_snsc_si = r_snsc * si.Ohm

        # Display input parameters with styling
        display(HTML('<div class="section-header">📊 Input Parameters</div>'))
        display(HTML(f'<div class="param-line">η = {eta}</div>'))
        display(HTML(f'<div class="param-line">n = {n}</div>'))
        display(
            HTML(f'<div class="param-line">α<sub>B</sub> = {alpha_B}</div>')
        )
        display(
            HTML(
                f'<div class="param-line">P<sub>BACKUP</sub> = '
                f"{p_backup_si}</div>"
            )
        )
        display(
            HTML(
                f'<div class="param-line">t<sub>BACKUP</sub> = '
                f"{t_BACKUP_si}</div>"
            )
        )
        display(
            HTML(
                f'<div class="param-line">V<sub>CELL(MAX)</sub> = '
                f"{v_cell_max_si}</div>"
            )
        )
        display(
            HTML(
                f'<div class="param-line">R<sub>SNSC</sub> = '
                f"{r_snsc_si}</div>"
            )
        )

        # Calculations
        display(
            HTML('<div class="section-header">🔬 Calculated Results</div>')
        )

        c_eol = (
            (4 * p_backup_si * t_BACKUP_si)
            / (n * eta * np.power(v_cell_max_si, 2))
            * np.power(
                alpha_B
                + np.sqrt(alpha_B)
                - (1 - alpha_B)
                * np.log(
                    (1 + np.sqrt(alpha_B)) / np.sqrt(1 - np.sqrt(alpha_B))
                ),
                -1,
            )
        )
        display(
            HTML(
                f'<div class="result-line"><strong>C<sub>EOL</sub>'
                f"</strong> = {c_eol}</div>"
            )
        )

        esr_eol = (
            eta
            * (1 - alpha_B)
            * n
            * np.power(v_cell_max_si, 2)
            / (4 * p_backup_si)
        )
        display(
            HTML(
                f'<div class="result-line"><strong>ESR<sub>EOL</sub>'
                f"</strong> = {esr_eol}</div>"
            )
        )

        v_stk_min__at_max_power = np.sqrt(4 * esr_eol * n * p_backup_si / eta)
        display(
            HTML(
                f'<div class="result-line"><strong>'
                f"V<sub>STK(MIN)@max_power</sub></strong> = "
                f"{v_stk_min__at_max_power}</div>"
            )
        )

        i_peak = 0.058 * si.V / r_snsc_si
        display(
            HTML(
                f'<div class="result-line"><strong>I<sub>PEAK</sub>'
                f"</strong> = {i_peak}</div>"
            )
        )

        v_stk_min__at_current_limit = (
            p_backup_si / (eta * i_peak)
        ) + n * esr_eol * i_peak
        display(
            HTML(
                f'<div class="result-line"><strong>'
                f"V<sub>STK(MIN)@current_limit</sub></strong> = "
                f"{v_stk_min__at_current_limit}</div>"
            )
        )

        v_stk_min = max(v_stk_min__at_max_power, v_stk_min__at_current_limit)
        display(
            HTML(
                f'<div class="result-line"><strong>V<sub>STK(MIN)</sub>'
                f"</strong> = {v_stk_min}</div>"
            )
        )


# Create interactive output
interactive_calc = widgets.interactive_output(
    calculate_and_display,
    {
        "eta": eta_slider,
        "n": n_slider,
        "alpha_B": alpha_B_slider,
        "p_backup": p_backup_slider,
        "t_BACKUP": t_backup_slider,
        "v_cell_max": v_cell_max_slider,
        "r_snsc": r_snsc_slider,
    },
)

# Display all widgets and output
display(
    widgets.VBox(
        [
            widgets.HTML(
                '<div class="calculator-header">'
                '<h2 style="margin:0; font-size:1.3em;">'
                "⚡ LTC3350 Interactive Calculator</h2></div>"
            ),
            eta_slider,
            n_slider,
            alpha_B_slider,
            p_backup_slider,
            t_backup_slider,
            v_cell_max_slider,
            r_snsc_slider,
            output,
        ],
        layout=widgets.Layout(
            padding="20px",
            width="100%",
            max_width="100%",
            overflow="hidden",
        ),
    )
)

# Trigger initial calculation
calculate_and_display(
    eta_slider.value,
    n_slider.value,
    alpha_B_slider.value,
    p_backup_slider.value,
    t_backup_slider.value,
    v_cell_max_slider.value,
    r_snsc_slider.value,
)
